**Contexte:** Actuellement les données APR (source Eclaireur) sont découpées par trajet et Tronço
- RDG ""Trajet"" :?
- RDG ""Tronçon"" : ensemble de données GPS avec même type de route ( autouroute, résidentielle,..) et même vitesse maximale (30,50,110,..)"							
							
**Le découpage par Tronçon** répondait à une vision très spécifique de la prévention routière. Pour monter des observatoires sur des zones de type : ville, arrondissement cette  notion n'est pas adaptée pour réaliser certaines analyses 							
							
Dans ce document nous recensons le nouveau découpage pour avoir plus précision sur les trajets réalisés : les rues, les infrastructures,... On va utiliser la notion "WAY" présente dans OSM							
							
**OpenStreetMap (OSM):**
- C'est une base de données géographiques libre
- Principes de base d'OpenStreetMap : nœuds et balises
**Overpass Turbo, un API:**
- c’est-à-dire une application servant à récolter des données OSM à partir de requêtes (usage avancé) :http://overpass-turbo.eu/
- overpass query language ( on peut interroger d'une maniere plus précise avec ce language ,par exemple on peut spécifier la date exacte..)
**Way:** est un ensemble des nodes

**Un way (chemin ou ligne )** est une suite de nœuds connectés comprenant entre 2 et 2000 noeuds qui permet de décrire des éléments comme des routes, des voies de chemin de fer, rivières, barrières, lignes électriques mais aussi des surfaces comme par exemple des plans d'eau ou des bâtiments. Un chemin peut faire partie d'une relation.                                                                                                                                           Chaque way est caractérisé de façon uniforme par des tags (attributs) qui permette de décrire ce qu'il représente, par exemple pour une rue, son nom, sa largeur, le revêtement, une limitation de vitesse, la présence d'une piste cyclable, etc.

**Les nœuds** sont définis par une latitude et une longitude (un unique point géospatial). Optionnellement, la troisième dimension, son altitude ou elevation en anglais, peut être indiquée à l’aide de l’attribut ele=*.

***Quelques règles géométriques concernant les chemins:***
- Plusieurs chemins peuvent passer par les mêmes nœuds,                                                      
- Un même chemin ne doit pas passer plusieurs fois par le même nœud (sauf premier et dernier nœud identiques d'un chemin fermé)    - Un chemin ne doit pas se couper lui-même.


**tags:** est balise contient key et value

In [0]:
from pyspark.sql.functions import *
import pyspark.sql.types as T
from pyspark.sql import Window

#### Partie I données 2022:

In [0]:
df_2022 = spark.read.parquet("/mnt/processed-prod/Daily/SpeedRoadType/2022/*/*")
df_2022=df_2022.orderBy("dateentry")

In [0]:
#filter highway (les routes et les routes de liasons)
df_2022=df_2022.filter(col("highway").isin(["motorway","trunk","primary","secondary","tertiary","residential","unclassified","motorway_link","trunk_link","primary_link","secondary_link","tertiary_link"]))

In [0]:
# def node_selection_f(node_list):
#   return node_list[-1]
# df_2022 = df_2022.withColumn('nodeId',node_selection_f(col("allnodes")))

In [0]:
def node_selection_f(node_list):
  """This function going to select the second node from all node (The most common one)"""  
  
  if (len(node_list) < 2):
    x= node_list[0]
  else:
    x= node_list[-1]
  return x


#The UDF to apply it on a pyspark dataframe
node_selection = udf(node_selection_f, T.LongType())

In [0]:
df_2022 = df_2022.withColumn('nodeId',node_selection(col("allnodes")))

In [0]:
#recuperer les node id et way id
def get_ways(year,spark):
  df_way = spark.read.parquet("/mnt/externaldata/france-"+year[2:]+"0101.osm.pbf.way.parquet")
  return df_way
df_ways=get_ways("2022",spark)
df_with_way = df_ways.select("id",explode("nodes").alias("nodes"))
df_with_nodes = df_with_way.select(col("id").alias("id_way"),col("nodes")["nodeId"].alias("nodeId"),col('nodes')["index"].alias("index"))

In [0]:
#jointure pour ajouter la colonne id way dans nos données de 2022
data_node_way_2022 = (df_2022.join(df_with_nodes.select("id_way","nodeId"), on= df_2022.nodeId == df_with_nodes.nodeId,how = "left").drop(df_with_nodes.nodeId))

In [0]:
#ajouter la colonne node d'intersection
df_nodes_count=data_node_way_2022.groupby("rideId","nodeId").agg(count_distinct("id_way").alias("count_way"))
df_nodes_count=df_nodes_count.withColumn("node_intersection",when(df_nodes_count.count_way>1,True).otherwise(False))

In [0]:
#jointure pour ajouter la colonne de node d'intersection 
data_final_node1 = (data_node_way_2022.join(df_nodes_count.select("nodeId","node_intersection"), on= df_2022.nodeId == df_nodes_count.nodeId,how = "left")
                            .drop(df_nodes_count.nodeId))

In [0]:
# drop les nodes que n'appartiennet à acun chemin qui presente 0.43% de nodes(735361)
 # sans id_way : 3222
data_final_node1=data_final_node1.dropna()

In [0]:
data_final_node1.write.option('header',True).mode('overwrite').parquet('/mnt/datalake/tmp/amani/data_node_way/way_2022')

### Partie II:

In [0]:
data=spark.read.parquet('/mnt/datalake/tmp/amani/data_node_way/way_2022')
data=data.orderBy("dateentry")


##### Duplicate

In [0]:
#si un trajet passe par un node d'intersection  on recupere le chemin celui d'avant 
#si un trejet qui commence par un node d'intersection on recupere celui d'aprés
#

In [0]:
duplicate = data.groupBy("deviceid","rideId","dateentry").count().filter(col("count")>1)

In [0]:
duplicate = duplicate.withColumn("duplicate",lit(True))
join_date = data.join(duplicate.select("deviceid","rideId","dateentry","duplicate"),["deviceid","rideId","dateentry"],"left")


In [0]:
join_data = join_date.withColumn("duplicate",when(col("duplicate")== True,True).otherwise(False))
join_data = join_data.withColumn("true_way",when(col("duplicate")==True,None).otherwise(col("id_way")))

In [0]:
join_data.count()

Out[48]: 412223708

In [0]:
join_data.filter(col("rideId")=="add9102d-04dd-4daa-a632-f9f8a79741f7_00c684595891aba0#77775073").select("id_way","duplicate","true_way").display()

id_way,duplicate,true_way
66731937,false,66731937
66731937,false,66731937
66731937,false,66731937
66731937,false,66731937
66731937,false,66731937
66731937,false,66731937
66731937,false,66731937
112515819,false,112515819
112515819,false,112515819
66731937,false,66731937


In [0]:
w = Window.partitionBy("rideId").orderBy(col("dateentry").asc())
join_data_2 = join_data.withColumn("true_way_2", last('true_way', True).over(w))

In [0]:
join_data_2.count()

Out[49]: 412223708

In [0]:
412223708


In [0]:
join_data_2.filter(col("rideId")=="add9102d-04dd-4daa-a632-f9f8a79741f7_00c684595891aba0#77775073").select("id_way","duplicate","true_way","true_way_2").display()

id_way,duplicate,true_way,true_way_2
112515770,true,null,null
112515819,true,null,null
112515770,true,null,null
112515819,true,null,null
112515819,false,112515819,112515819
112515819,false,112515819,112515819
112515819,false,112515819,112515819
112515819,false,112515819,112515819
112515819,false,112515819,112515819
364370751,true,null,112515819


In [0]:
w = Window.partitionBy("rideId").orderBy(col("dateentry").desc())
join_data_3 = join_data_2.withColumn("true_way_3", last('true_way_2', True).over(w))

In [0]:
join_data_3.count()

Out[51]: 412223708

In [0]:
join_data_3.filter(col("rideId")=="add9102d-04dd-4daa-a632-f9f8a79741f7_00c684595891aba0#77775073").select("id_way","duplicate","true_way","true_way_2","true_way_3").display()

id_way,duplicate,true_way,true_way_2,true_way_3
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739


In [0]:
#Drop duplicate rows
df=join_data_3.dropDuplicates(["deviceid","rideId","dateentry"])

In [0]:
df.filter(col("rideId")=="add9102d-04dd-4daa-a632-f9f8a79741f7_00c684595891aba0#77775073").select("id_way","duplicate","true_way","true_way_2","true_way_3").display()

id_way,duplicate,true_way,true_way_2,true_way_3
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
135670442,true,null,641352739,641352739
4652238,true,null,641352739,641352739
4652238,true,null,641352739,641352739


In [0]:
df_way_null=df.filter(col("true_way_3").isNull())
df_way_null.select("nodeId").distinct().count()

Out[36]: 63944

In [0]:
df_way_null.display()

deviceid,rideId,dateentry,latitude,longitude,altitude,bearing,speed,acceleration,accuracy,lat,lng,node_start,node_end,allnodes,nearestcalled,highway,maxspeed,id_way,nodeId,node_intersection,duplicate,true_way,true_way_2,true_way_3
01f9ce7f27a4a718#tkgoji10,b9c11c13-e66f-4a99-bd60-864d40767907_01f9ce7f27a4a718#tkgoji10,1641837733000,50.4617239,2.8529928,90.70000457763672,71.68478,6.60116753825435,-1.3335675597928587,3.9,50.461723,2.852993,528711425,528711437,"List(528711425, 528711437)",false,unclassified,70,42349386,528711437,true,true,null,null,null
01f9ce7f27a4a718#tkgoji10,b9c11c13-e66f-4a99-bd60-864d40767907_01f9ce7f27a4a718#tkgoji10,1641837734000,50.4617302,2.8530795,90.70000457763672,86.68802,6.223961122311333,-0.37720641594301707,3.9,50.461735,2.853079,528711425,528711437,"List(528711425, 528711437)",false,unclassified,70,42349386,528711437,true,true,null,null,null
01f9ce7f27a4a718#tkgoji10,b9c11c13-e66f-4a99-bd60-864d40767907_01f9ce7f27a4a718#tkgoji10,1641837735000,50.4617235,2.8531572,90.70000457763672,92.89966,1.2711077534219637,-4.952853368889369,3.9,50.46174,2.853096,528711425,528711437,"List(528711425, 528711437)",false,unclassified,70,42349386,528711437,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893938044,43.701142825532926,1.8265390558851167,133.44746057968587,96.3183364868164,7.488509456572747,-0.040942205111073486,4.0,43.70114,1.826539,443020613,443020614,"List(443020613, 443020614)",false,residential,70,37767831,443020614,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893939052,43.70112059919265,1.8267576249207322,134.18227740749717,96.71483612060547,17.59070239249159,10.022016801506789,4.0,43.701122,1.826758,443020613,443020614,"List(443020613, 443020614)",false,residential,70,37767831,443020614,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893940044,43.70110485982943,1.8269796730168864,134.5722597539425,96.71483612060547,18.114227529798256,0.5277471142204302,4.0,43.701103,1.82698,443020613,443020614,"List(443020613, 443020614)",false,residential,70,37767831,443020614,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893941038,43.701085301025785,1.8271996598692133,134.6897033378482,96.71541595458984,17.924618481082955,-0.19075357013611774,4.0,43.701084,1.8272,443020613,443020614,"List(443020613, 443020614)",false,residential,70,37767831,443020614,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893942044,43.701069246774374,1.8274081169976275,134.00467489566654,96.71484375,16.67087496776385,-1.2462659178122313,4.0,43.701065,1.827407,443020613,443020614,"List(443020613, 443020614)",false,residential,70,37767831,443020614,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893943043,43.70105075221606,1.8276093300921645,134.15501480270177,96.71484375,16.31202207551582,-0.3592121043523831,4.0,43.701046,1.827608,443020613,443020614,"List(443020613, 443020614, 775945259)",false,residential,70,33136267,775945259,true,true,null,null,null
035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213,1643893944044,43.70103219774394,1.8278062701674904,133.64713524188846,95.11061096191406,15.96624934682009,-0.3454273013943354,4.0,43.701027,1.827805,443020614,775945259,"List(443020614, 775945259)",false,tertiary,50,33136267,775945259,true,true,null,null,null


In [0]:
df.filter(col("rideId")=="b9c11c13-e66f-4a99-bd60-864d40767907_01f9ce7f27a4a718#tkgoji10").select("id_way","duplicate","true_way","true_way_2","true_way_3").display()

id_way,duplicate,true_way,true_way_2,true_way_3
42349386,true,null,null,null
42349386,true,null,null,null
42349386,true,null,null,null


In [0]:
df.filter(col("rideId")=="852cc81a-ff3d-41fa-91f8-f6898c551a2a_035ED254-AEA1-4EA3-84EE-37A05A09032B#81597213").select("id_way","duplicate","true_way","true_way_2","true_way_3").display()

id_way,duplicate,true_way,true_way_2,true_way_3
203240816,true,null,null,null
203240816,true,null,null,null
203240816,true,null,null,null
203240816,true,null,null,null
203240816,true,null,null,null
203240816,true,null,null,null
33136267,true,null,null,null
33136267,true,null,null,null
33136267,true,null,null,null
33136267,true,null,null,null


In [0]:
df.select("nodeId").distinct().count()

Out[37]: 478940

In [0]:
(63944/478940)*100

Out[38]: 13.351150457259783

In [0]:
#grouper par chemin en gardant la liste de node a fin d'ajouter ID 
df_way=df.groupby("rideId","true_way_3").agg(collect_set('nodeId').alias("list_nodes"))

In [0]:
df_way=df_way.withColumn("tronçon_physique", monotonically_increasing_id())
df_way=df_way.withColumn("tronçon_physique2", concat(col("true_way_3"),lit("#"),col("tronçon_physique"))) 

#### tronçons physiques

In [0]:
#jointure final
data_tronçons = df.join(df_way, ["rideId","true_way_3"],"left")

In [0]:
data_tronçons.write.option('header',True).mode('overwrite').parquet('/mnt/datalake/tmp/amani/data_node_way/tronçons_physiques_2022')

### Partie III

In [0]:
from pyspark.sql.functions import *
import pyspark.sql.types as T
from pyspark.sql import Window

In [0]:
def get_ways_with_keyvalues(df_way,keyvalues):
  #On crée une ligne par tag
  df_way2= df_way.select(df_way.id,df_way.nodes,explode(df_way.tags).alias("tags"))
  
  #On récupère le contenu des tags pour avoir les colonnes id, key et value  
  df_way3=df_way2.select(df_way2.id,df_way2.nodes,
                         df_way2.tags["key"].cast("string").alias("key"),
                         df_way2.tags["value"].cast("string").alias("value"))
  
  #On filtre les lignes pour garder que les lignes dont les keys ou les vavlues ont le mot cycleway par exemple
  #pour le premier valeur de keyvalues tient pas en compte
  #on recupere le premier, apres on l'ajoute dans la BOUCLE
  df_way4=df_way3.filter(df_way3.key.contains(keyvalues[0]["key"]) & df_way3.value.contains(keyvalues[0]["value"]))
  
  for x in keyvalues[1:]:
    df_way4=df_way4.union(df_way3.filter(df_way3.key.contains(x["key"]) & df_way3.value.contains(x["value"])))
    
  return df_way4

In [0]:
def get_ways(year,spark):
  df_way = spark.read.parquet("/mnt/externaldata/france-"+year[2:]+"0101.osm.pbf.way.parquet")
  return df_way
df_ways=get_ways("2022",spark)

In [0]:
kv_ = [{"key":"","value":""}]
df_type_way=get_ways_with_keyvalues(df_ways,kv_)
df_type_way.display()

id,nodes,key,value
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",cycleway:right,no
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",foot,no
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",highway,tertiary
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",lane_markings,no
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",lit,yes
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",maxspeed,30
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",name,Rue Nationale
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",oneway,yes
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",sidewalk,none
2569,"List(List(0, 382017), List(1, 3745434701), List(2, 1836953770), List(3, 3745434708), List(4, 505820718))",surface,asphalt


In [0]:
data_tronçons = spark.read.parquet('/mnt/datalake/tmp/amani/data_node_way/tronçons_physiques_2022')
data_tronçons =data_tronçons.orderBy("dateentry")

In [0]:
data_tronçons.select(col("tronçon_physique2")).distinct().count()

Out[16]: 53296

In [0]:
data_type_way = (data_tronçons.join(df_type_way.select("id","key","value"), on= data_tronçons.true_way_3== df_type_way.id,how = "left").drop(df_type_way.id))


In [0]:
data_type_way.count()

Out[13]: 46576859

In [0]:
 (36932/ 67343922)*100    

In [0]:
key_null=data_type_way.filter(col("key").isNull())

In [0]:
key_null.count()

Out[8]: 2091445

In [0]:
key_null.select(col("true_way_3")).distinct().count()

Out[9]: 4

In [0]:
key_null.groupby(col("true_way_3")).count().display()

true_way_3,count
null,2089948
745396659,14
382135460,1469
807838385,14


In [0]:
key_null.select("rideId").distinct().count()

Out[26]: 11880

In [0]:
data_type_way.select("rideId").distinct().count()

Out[27]: 22169

In [0]:
#trajet
(11880/22169)*100


Out[28]: 53.58834408408137

In [0]:
key_null.select("nodeId").distinct().count()

Out[18]: 63960

In [0]:
data_type_way.select("nodeId").distinct().count()

Out[19]: 473450

In [0]:
(63960/473450)*100

Out[20]: 13.509346287886789

#### filtrer data

In [0]:
data_type_way=data_type_way.filter(~col("key").isNull())

In [0]:
data_type_way.display()

rideId,true_way_3,deviceid,dateentry,latitude,longitude,altitude,bearing,speed,acceleration,accuracy,lat,lng,node_start,node_end,allnodes,nearestcalled,highway,maxspeed,id_way,nodeId,node_intersection,duplicate,true_way,true_way_2,list_nodes,tronçon_physique,tronçon_physique2,key,value
80abb0ea-dfcc-46a5-b07f-a3b0b82a9a94_bdb0d7b6302464b4#96834456,1631480,bdb0d7b6302464b4#96834456,1660386219000,48.7384,-3.4391424,88.78778076171875,340.95218,23.438745145785745,0.4789205237592178,3.9,48.73839,-3.439169,6993281,5181259615,"List(6993281, 5181259615, 6992075)",false,primary,80,1631480,6992075,false,false,1631480,1631480,"List(4731300652, 6985913, 6992185, 5181264329, 6993058, 5181264323, 4731286840, 5181264322, 6986699, 6993037, 5181264324, 6992581, 4731300651, 6993126, 6993178, 6992075, 5181264327, 6985861, 3038120259, 6992321, 6993410, 4731300653)",8589936155,1631480#8589936155,class:bicycle:commute,-2
80abb0ea-dfcc-46a5-b07f-a3b0b82a9a94_bdb0d7b6302464b4#96834456,1631480,bdb0d7b6302464b4#96834456,1660386219000,48.7384,-3.4391424,88.78778076171875,340.95218,23.438745145785745,0.4789205237592178,3.9,48.73839,-3.439169,6993281,5181259615,"List(6993281, 5181259615, 6992075)",false,primary,80,1631480,6992075,false,false,1631480,1631480,"List(4731300652, 6985913, 6992185, 5181264329, 6993058, 5181264323, 4731286840, 5181264322, 6986699, 6993037, 5181264324, 6992581, 4731300651, 6993126, 6993178, 6992075, 5181264327, 6985861, 3038120259, 6992321, 6993410, 4731300653)",8589936155,1631480#8589936155,highway,primary
80abb0ea-dfcc-46a5-b07f-a3b0b82a9a94_bdb0d7b6302464b4#96834456,1631480,bdb0d7b6302464b4#96834456,1660386219000,48.7384,-3.4391424,88.78778076171875,340.95218,23.438745145785745,0.4789205237592178,3.9,48.73839,-3.439169,6993281,5181259615,"List(6993281, 5181259615, 6992075)",false,primary,80,1631480,6992075,false,false,1631480,1631480,"List(4731300652, 6985913, 6992185, 5181264329, 6993058, 5181264323, 4731286840, 5181264322, 6986699, 6993037, 5181264324, 6992581, 4731300651, 6993126, 6993178, 6992075, 5181264327, 6985861, 3038120259, 6992321, 6993410, 4731300653)",8589936155,1631480#8589936155,lanes,2
80abb0ea-dfcc-46a5-b07f-a3b0b82a9a94_bdb0d7b6302464b4#96834456,1631480,bdb0d7b6302464b4#96834456,1660386219000,48.7384,-3.4391424,88.78778076171875,340.95218,23.438745145785745,0.4789205237592178,3.9,48.73839,-3.439169,6993281,5181259615,"List(6993281, 5181259615, 6992075)",false,primary,80,1631480,6992075,false,false,1631480,1631480,"List(4731300652, 6985913, 6992185, 5181264329, 6993058, 5181264323, 4731286840, 5181264322, 6986699, 6993037, 5181264324, 6992581, 4731300651, 6993126, 6993178, 6992075, 5181264327, 6985861, 3038120259, 6992321, 6993410, 4731300653)",8589936155,1631480#8589936155,maxspeed,80
80abb0ea-dfcc-46a5-b07f-a3b0b82a9a94_bdb0d7b6302464b4#96834456,1631480,bdb0d7b6302464b4#96834456,1660386219000,48.7384,-3.4391424,88.78778076171875,340.95218,23.438745145785745,0.4789205237592178,3.9,48.73839,-3.439169,6993281,5181259615,"List(6993281, 5181259615, 6992075)",false,primary,80,1631480,6992075,false,false,1631480,1631480,"List(4731300652, 6985913, 6992185, 5181264329, 6993058, 5181264323, 4731286840, 5181264322, 6986699, 6993037, 5181264324, 6992581, 4731300651, 6993126, 6993178, 6992075, 5181264327, 6985861, 3038120259, 6992321, 6993410, 4731300653)",8589936155,1631480#8589936155,oneway,no
80abb0ea-dfcc-46a5-b07f-a3b0b82a9a94_bdb0d7b6302464b4#96834456,1631480,bdb0d7b6302464b4#96834456,1660386219000,48.7384,-3.4391424,88.78778076171875,340.95218,23.438745145785745,0.4789205237592178,3.9,48.73839,-3.439169,6993281,5181259615,"List(6993281, 5181259615, 6992075)",false,primary,80,1631480,6992075,false,false,1631480,1631480,"List(4731300652, 6985913, 6992185, 5181264329, 6993058, 5181264323, 4731286840, 5181264322, 6986699, 6993037, 5181264324, 6992581, 4731300651, 6993126, 6993178, 6992075, 5181264327, 6985861, 3038120259, 6992321, 6993410, 4731300653)",8589936155,1631480#8589936155,ref,D 767
80abb0ea-dfcc-46a5-b07f-a3

#### Partie IIII calcule la median

In [0]:
from pyspark.sql import DataFrame

class median():
    """ Create median class with over method to pass partition """
    def __init__(self, df, col, name):
        assert col
        self.column=col
        self.df = df
        self.name = name

    def over(self, window):
        from pyspark.sql.functions import percent_rank, pow, first

        first_window = window.orderBy(self.column)                                  # first, order by column we want to compute the median for
        df = self.df.withColumn("percent_rank", percent_rank().over(first_window))  # add percent_rank column, percent_rank = 0.5 coressponds to median
        second_window = window.orderBy(pow(df.percent_rank-0.5, 2))                 # order by (percent_rank - 0.5)^2 ascending
        return df.withColumn(self.name, first(self.column).over(second_window))     # the first row of the window corresponds to median

def addMedian(self, col, median_name):
    """ Method to be added to spark native DataFrame class """
    return median(self, col, median_name)

# Add method to DataFrame class
DataFrame.addMedian = addMedian

In [0]:
# import statistics
# from pyspark.sql.types import DoubleType

# median_udf = udf(lambda x: statistics.median(x) if bool(x) else None, DoubleType())

# #... .groupBy('something').agg(median_udf(F.collect_list(F.col('value'))).alias('median'))

In [0]:
median_window =  Window.partitionBy("tronçon_physique2").orderBy(col("dateentry").asc())
data_type_way= (data_type_way.addMedian("nodeId", "median_node").over(median_window)
                             .addMedian("lat", "median_lat").over(median_window)
                             .addMedian("lng", "median_lng").over(median_window))
                           

In [0]:
data_type_way.select(col("tronçon_physique2")).distinct().count()

Out[15]: 53292

### Groupby

In [0]:
data_final=data_type_way.groupby("tronçon_physique2","key","value").agg(
                                                                                    first("nodeId").alias('node_start'),
                                                                              first("median_node").alias('node_central'),
                                                                              last("nodeId").alias('node_end'),
                                                                              first("lat").alias('lat_start'),
                                                                              first("lng").alias('lng_start'),
                                                                              first( "median_lat").alias("median_lat"),
                                                                              first( "median_lng").alias("median_lng"),
                                                                              last("lat").alias('lat_end'),
                                                                              last("lng").alias('lng_end'),
                                                                              avg("speed").alias("vitesse_moy"),
                                                                              avg("acceleration").alias('acceleration_moy'))

In [0]:
data_final.display()

tronçon_physique2,key,value,node_start,node_central,node_end,lat_start,lng_start,median_lat,median_lng,lat_end,lng_end,vitesse_moy,acceleration_moy
100056673#77309412081,highway,primary_link,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100056673#77309412081,lanes,1,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100056673#77309412081,maxspeed,70,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100056673#77309412081,oneway,yes,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100056673#77309412081,source,cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2009,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100056673#77309412081,source:maxspeed,sign,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100056673#77309412081,surface,asphalt,469750289,469750289,469750289,47.943504,1.888707,47.943504,1.888707,47.943504,1.888707,5.418772981339165,0.09184360985320619
100147588#68719478037,cycleway:both,no,2638433598,242195334,242195262,45.635574,-0.975023,45.635574,-0.975023,45.63163,-0.986875,13.002854422962274,0.15359629043986212
100147588#68719478037,highway,primary,2638433598,242195334,242195262,45.635574,-0.975023,45.635574,-0.975023,45.63163,-0.986875,13.002854422962274,0.15359629043986212
100147588#68719478037,lanes,2,2638433598,242195334,242195262,45.635574,-0.975023,45.635574,-0.975023,45.63163,-0.986875,13.002854422962274,0.15359629043986212


In [0]:
data_final.write.option('header',True).mode('overwrite').parquet('/mnt/datalake/tmp/amani/data_node_way/tronçons_physiques_2022_type')